# Docker Swarm startup

This example runs a [Docker Swarm Mode](https://docs.docker.com/engine/swarm/key-concepts/) swarm with one manager and one worker node.

The example needs to configure [ssh-keys](https://www.ssh.com/ssh/key/#how-to-configure-key-based-authentication) for the manager and worker. Keys can be generated with the Notebook, and copying them to manager and worker node requires authentication for the first time. The passwords needed for that are read from files `manager.password` and `worker.password` in the work folder. __Please create those__.

## Installing required packages
- Ssh-client is required for connecting to manager and worker hosts.
- Expect is used for entering password while adding the ssh-keys

In [ ]:
# Install requirements

!sudo apt -y update && sudo apt -y install ssh-client expect

# TODO later we will likely need python packages too. Below is an example how to install them.
#import sys
#!conda install --yes --prefix {sys.prefix} --file requirements.txt

## Configuration variables
`MANAGER_IP` and `WORKER_IP` addresses are needed and must be accessible from network.

Passwords have to be in files `manager.password` and `worker.password` which are ignored from version control.

In [ ]:
MANAGER_IP='10.0.2.69'
MANAGER_SSH_USER='petrisi'
outputs=!cat manager.password
MANAGER_PASSWORD=outputs[0]
WORKER_IP='10.0.2.68'
WORKER_SSH_USER='petrisi'
outputs=!cat worker.password
WORKER_PASSWORD=outputs[0]

## Environment configuration
Install a ssh-config to skip strict host key checks on MANAGER_IP and WORKER_IP connections.

In [ ]:
![ ! -e $HOME/.ssh/mykey ] && ssh-keygen -N "" -C "jupyternb@$(hostname)" -f $HOME/.ssh/mykey

In [ ]:
%%script env MANAGER_IP="$MANAGER_IP" WORKER_IP="$WORKER_IP" MANAGER_SSH_USER="$MANAGER_SSH_USER" WORKER_SSH_USER="$WORKER_SSH_USER" bash

cat << EOF >$HOME/.ssh/config
Host $MANAGER_IP $WORKER_IP
    StrictHostKeyChecking no
    IdentityFile ~/.ssh/mykey
Host $MANAGER_IP
    User $MANAGER_SSH_USER
Host $WORKER_IP
    User $WORKER_SSH_USER
EOF

Generate the ssh-keys

Copy ssh-key to MANAGER host.

In [ ]:
%%script env MY_PASSWORD="$MANAGER_PASSWORD" MANAGER_IP="$MANAGER_IP" MANAGER_SSH_USER="$MANAGER_SSH_USER" bash

cat << EOF | /usr/bin/expect -
set timeout 1
spawn ssh-copy-id -i $HOME/.ssh/mykey ${MANAGER_SSH_USER}@${MANAGER_IP}
expect "Password:"
send "$MY_PASSWORD\n"
expect eof
EOF

Copy ssh-key to WORKER host.

In [ ]:
%%script env MY_PASSWORD="$WORKER_PASSWORD" WORKER_IP="$WORKER_IP" WORKER_SSH_USER="$WORKER_SSH_USER" bash

cat << EOF | /usr/bin/expect -
set timeout 1
spawn ssh-copy-id -i $HOME/.ssh/mykey ${WORKER_SSH_USER}@${WORKER_IP}
expect "Password:"
send "$MY_PASSWORD\n"
expect eof
EOF

Test the connection

In [ ]:
!ssh $MANAGER_IP docker ps

In [ ]:
!ssh $WORKER_IP docker ps

In [ ]:
def on_manager(command):
    rval = !ssh $MANAGER_IP $command
    return rval[0]

def on_worker(command):
    rval = !ssh $WORKER_IP $command
    return rval[0]
    
on_manager(f'docker swarm init --advertise-addr {MANAGER_IP}')
worker_join_token = on_manager(f'docker swarm join-token -q worker')

on_worker(f'docker swarm join --token {worker_join_token} {MANAGER_IP}:2377')